<a href="https://colab.research.google.com/github/Far-ch/Signals-and-Systems-Project/blob/main/Signal_and_Systems_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#YOLO
from ultralytics import YOLO
import cv2, torch, numpy as np, matplotlib.pyplot as plt

VIDEO_IN  = "person1.mp4"
SNAPSHOT  = "first_detected.jpg"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL  = "yolov8m.pt"
model  = YOLO(MODEL).to(DEVICE)
CLASSES = {
    0: "person",1: "bicycle",2: "car",3: "motorcycle",4: "airplane",5: "bus",6: "train",7: "truck",8: "boat",9: "traffic light",10: "fire hydrant",
11: "stop sign", 12: "parking meter",13: "bench",14: "bird",15: "cat",16: "dog",17: "horse",18: "sheep",19: "cow",20: "elephant",21: "bear",
22: "zebra",23: "giraffe",24: "backpack",25: "umbrella",26: "handbag",27: "tie",28: "suitcase",29: "frisbee",30: "skis",31: "snowboard",32: "sports ball",
33: "kite",34: "baseball bat",35: "baseball glove",36: "skateboard",37: "surfboard",38: "tennis racket",39: "bottle",40: "wine glass",41: "cup",
42: "fork",43: "knife",44: "spoon",45: "bowl",46: "banana",47: "apple",48: "sandwich",49: "orange",50: "broccoli",51: "carrot",
52: "hot dog", 53: "pizza",54: "donut",55: "cake",56: "chair",57: "couch",58: "potted plant",59: "bed",60: "dining table",61: "toilet",
62: "tv",63: "laptop",64: "mouse",65: "remote",66: "keyboard",67: "cell phone",68: "microwave",69: "oven",70: "toaster",71: "sink",72: "refrigerator",
73: "book",74: "clock",75: "vase",76: "scissors",77: "teddy bear",78: "hair drier",79: "toothbrush"}

CLASS_IDS = list(CLASSES.keys())
rng = np.random.default_rng(42)
COLORS = {cls_id: tuple(rng.integers(64, 256, 3).tolist()) for cls_id in CLASS_IDS}

BOX_THICK = 3
FONT_SCALE = 1.0
FONT_THICK = 2


def detect(img_rgb, imgsz=960, conf=0.3):
    """Detect objects and return list of boxes: (x1, y1, x2, y2, class_id, conf)"""
    res = model.predict(img_rgb, imgsz=imgsz, conf=conf,
                        classes=CLASS_IDS, device=DEVICE,
                        verbose=False)[0]
    out = []
    for b in res.boxes:
        out.append((*map(int, b.xyxy[0]), int(b.cls[0]), float(b.conf[0])))
    return out


def robust_detect(frame_bgr):
    H, W = frame_bgr.shape[:2]

    #Full-frame
    boxes = detect(cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB), 1280, 0.3)
    if boxes:
        return boxes

    #Center crop + upscale
    cw, ch = int(W * 0.6), int(H * 0.7)
    x0, y0 = (W - cw) // 2, (H - ch) // 2
    crop = frame_bgr[y0:y0+ch, x0:x0+cw]
    crop_up = cv2.resize(crop, (1280, 1280))
    boxes_crop = detect(cv2.cvtColor(crop_up, cv2.COLOR_BGR2RGB), 1280, 0.25)
    if boxes_crop:
        sx, sy = cw / 1280, ch / 1280
        return [(int(x1*sx + x0), int(y1*sy + y0),
                 int(x2*sx + x0), int(y2*sy + y0), cls, conf)
                for x1, y1, x2, y2, cls, conf in boxes_crop]

    #Sliding tiles
    out = []
    tile, stride = 640, 320
    for y in range(0, H, stride):
        for x in range(0, W, stride):
            tile_bgr = frame_bgr[y:y+tile, x:x+tile]
            boxes_tile = detect(cv2.cvtColor(tile_bgr, cv2.COLOR_BGR2RGB), 640, 0.25)
            for x1, y1, x2, y2, cls, conf in boxes_tile:
                out.append((x1+x, y1+y, x2+x, y2+y, cls, conf))
    return out


cap = cv2.VideoCapture(VIDEO_IN)
ok, frame = cap.read()
cap.release()
assert ok, f" Couldn't read first frame of {VIDEO_IN}"

boxes = robust_detect(frame.copy())
for x1, y1, x2, y2, cls, conf in boxes:
    label_name = CLASSES.get(cls, "unknown")
    label_text = f"{label_name} {conf:.2f}"

    color = COLORS.get(cls, (0, 255, 255))
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, BOX_THICK)

    (tw, th), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX,
                                  FONT_SCALE, FONT_THICK)
    cv2.rectangle(frame, (x1, y1 - th - 8), (x1 + tw, y1), color, -1)
    cv2.putText(frame, label_text, (x1, y1 - 5),
                cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE,
                (255, 255, 255), FONT_THICK, cv2.LINE_AA)


cv2.imwrite(SNAPSHOT, frame)
plt.figure(figsize=(12,6))
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

In [ ]:
#Fast-R-CNN
!pip -q install --upgrade torch torchvision torchaudio
import cv2, torch, matplotlib.pyplot as plt
import numpy as np
from torchvision import models, transforms

# COCO classes
CLASSES = {
    0:"background", 1:"person", 2:"bicycle", 3:"car", 4:"motorcycle", 5:"airplane",
    6:"bus",7:"train",8:"truck",9:"boat",10:"trafficlight",11:"firehydrant",
    13:"bench",14:"bird",15:"cat",16:"dog",17:"horse",18:"sheep",19:"cow",
    20:"elephant",21:"bear",22:"zebra",23:"giraffe",24:"backpack",25:"umbrella",
    27:"handbag",28:"tie",31:"snowboard",32:"sportsball",33:"kite",34:"baseballbat",
    35:"baseballglove",36:"skateboard",37:"surfboard",38:"tennisracket",39:"bottle",
    40:"wineglass",41:"cup",42:"fork",43:"knife",44:"spoon",45:"bowl",46:"banana",
    47:"apple",48:"sandwich",49:"orange",50:"broccoli",51:"carrot",52:"hotdog",
    53:"pizza",54:"donut",55:"cake",56:"chair",57:"couch",58:"pottedplant",59:"bed",
    60:"diningtable",61:"toilet",62:"tv",63:"laptop",64:"mouse",65:"remote",
    66:"keyboard",67:"cellphone",68:"microwave",69:"oven",70:"toaster",71:"sink",
    72:"refrigerator",73:"book",74:"clock",75:"vase",76:"scissors",77:"teddybear",
    78:"hairdrier",79:"toothbrush"
}

VIDEO_PATH = "person1.mp4"
cap = cv2.VideoCapture(VIDEO_PATH)
ok, frame_bgr = cap.read()
cap.release()
assert ok, "Could not read first frame"

frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

device = "cuda" if torch.cuda.is_available() else "cpu"
model  = models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
model.to(device).eval()

transform = transforms.Compose([transforms.ToTensor()])   # converts to [0,1] + CHW
img_tensor = transform(frame_rgb).to(device)


with torch.no_grad():
    preds = model([img_tensor])[0]   # dict with boxes, labels, scores

boxes   = preds["boxes"].cpu().numpy()
labels  = preds["labels"].cpu().numpy()
scores  = preds["scores"].cpu().numpy()

CONF_TH = 0.50
keep = scores >= CONF_TH

boxes, labels, scores = boxes[keep], labels[keep], scores[keep]

OUT = frame_bgr.copy()
for (x1,y1,x2,y2), cls_id, conf in zip(boxes, labels, scores):
    cls_id = int(cls_id)
    name   = CLASSES.get(cls_id, f"id{cls_id}")
    label  = f"{name} {conf:.2f}"

    color = (0,255,0)   # green boxes; change if you like
    cv2.rectangle(OUT, (int(x1),int(y1)), (int(x2),int(y2)), color, 2)
    (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
    cv2.rectangle(OUT, (int(x1), int(y1)-th-8), (int(x1)+tw, int(y1)), color, -1)
    cv2.putText(OUT, label, (int(x1), int(y1)-4),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 2)

cv2.imwrite("fast_rcnn_detect.jpg", OUT[:, :, ::-1])  # BGR→RGB file

plt.figure(figsize=(12,6))
plt.imshow(cv2.cvtColor(OUT, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Fast-R-CNN detection ")
plt.show()


In [ ]:
#MOSSE_Tracker
import time
import uuid
import cv2
import numpy as np
import torch
from torchvision import models, transforms
from pathlib import Path

#PATHS
downloads = Path.home() / "Downloads"
VIDEO_PATH = downloads / "videos" / "car1.mp4"
OUTPUT_PATH = downloads / "car1_mosse_final_output.mp4"

#PARAMETERS
CONF_TH = 0.5
MAX_LOST = 250
HIST_THRESH = 0.6
HIST_BINS = (16, 16, 16)

#COCO LABELS
COCO = {i: name for i, name in enumerate([
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', '', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', '', 'backpack', 'umbrella', '', '',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
    'tennis racket', 'bottle', '', 'wine glass', 'cup', 'fork', 'knife',
    'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli',
    'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
    'potted plant', 'bed', '', 'dining table', '', '', 'toilet', '', 'tv',
    'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
    'oven', 'toaster', 'sink', 'refrigerator', '', 'book', 'clock', 'vase',
    'scissors', 'teddy bear', 'hair drier', 'toothbrush'])}

#Histogram functions
def get_histogram(patch):
    hsv = cv2.cvtColor(patch, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, HIST_BINS, [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

def compare_hist(hist1, hist2):
    return cv2.compareHist(hist1.astype(np.float32), hist2.astype(np.float32), cv2.HISTCMP_CORREL)

#Kalman Filter
class KalmanFilter:
    def __init__(self, x, y):
        self.kf = cv2.KalmanFilter(8, 4)
        self.kf.transitionMatrix = np.eye(8, dtype=np.float32)
        for i in range(4):
            self.kf.transitionMatrix[i, i + 4] = 1
        self.kf.measurementMatrix = np.zeros((4, 8), np.float32)
        for i in range(4):
            self.kf.measurementMatrix[i, i] = 1
        self.kf.processNoiseCov = np.eye(8, dtype=np.float32) * 1e-3
        self.kf.measurementNoiseCov = np.eye(4, dtype=np.float32) * 1e-2
        self.kf.statePost = np.array([[x], [y], [0], [0], [0], [0], [0], [0]], np.float32)

    def predict(self):
        pred = self.kf.predict()
        return int(pred[0, 0]), int(pred[1, 0])

    def correct(self, x, y):
        self.kf.correct(np.array([[x], [y], [0], [0]], np.float32))

#Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT").to(device).eval()
transform = transforms.Compose([transforms.ToTensor()])

#Load video
cap = cv2.VideoCapture(str(VIDEO_PATH))
assert cap.isOpened(), f"Cannot open video: {VIDEO_PATH}"
W, H = int(cap.get(3)), int(cap.get(4))
fps_in = cap.get(cv2.CAP_PROP_FPS) or 25
out = cv2.VideoWriter(str(OUTPUT_PATH), cv2.VideoWriter_fourcc(*'mp4v'), fps_in, (W, H))

#Initialize
tracks = []
frame_idx = 0
start_time = time.time()
ret, frame = cap.read()
frame_idx += 1
assert ret, "Can't read the first frame"
img_tensor = transform(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).to(device)

with torch.no_grad():
    preds = model([img_tensor])[0]

boxes = preds["boxes"].cpu().numpy()
labels = preds["labels"].cpu().numpy()
scores = preds["scores"].cpu().numpy()

for box, cls, score in zip(boxes, labels, scores):
    if score < CONF_TH:
        continue
    x1, y1, x2, y2 = map(int, box)
    w, h = x2 - x1, y2 - y1
    patch = frame[y1:y2, x1:x2]
    if patch.size == 0:
        continue
    tracker = cv2.legacy.TrackerMOSSE_create()
    tracker.init(frame, (x1, y1, w, h))
    kalman = KalmanFilter(x1 + w // 2, y1 + h // 2)
    hist = get_histogram(patch)
    label = COCO.get(int(cls), f"class{int(cls)}")
    tracks.append({
        "id": str(uuid.uuid4())[:8],
        "tracker": tracker,
        "label": label,
        "kalman": kalman,
        "hist": hist,
        "lost": 0
    })
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

out.write(frame)

#Track across video
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1
    new_tracks = []

    for tr in tracks:
        ok, bbox = tr["tracker"].update(frame)
        x, y, w, h = map(int, bbox)
        x2, y2 = x + w, y + h

        if not ok or w <= 0 or h <= 0:
            tr["lost"] += 1
            pred_x, pred_y = tr["kalman"].predict()
            if tr["lost"] <= MAX_LOST:
                cv2.circle(frame, (pred_x, pred_y), 4, (255, 255, 0), -1)
                new_tracks.append(tr)
            continue

        patch = frame[y:y+h, x:x+w]
        if patch.size > 0:
            hist_new = get_histogram(patch)
            sim = compare_hist(tr["hist"], hist_new)
            if sim >= HIST_THRESH:
                tr["kalman"].correct(x + w // 2, y + h // 2)
                tr["hist"] = 0.8 * tr["hist"] + 0.2 * hist_new
                tr["lost"] = 0
            else:
                tr["lost"] += 1
                if tr["lost"] <= MAX_LOST:
                    pred_x, pred_y = tr["kalman"].predict()
                    cv2.circle(frame, (pred_x, pred_y), 4, (255, 0, 0), -1)
                    new_tracks.append(tr)
                continue

        cv2.rectangle(frame, (x, y), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, tr["label"], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        new_tracks.append(tr)

    tracks = new_tracks

    elapsed = time.time() - start_time
    fps = frame_idx / elapsed
    cv2.putText(frame, f"FPS: {fps:.1f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    out.write(frame)


cap.release()
out.release()
print(f"✅ Done. Output saved to: {OUTPUT_PATH}")
print(f"📈 Average FPS: {frame_idx / (time.time() - start_time):.2f}")

In [ ]:
#CSRT_Tracker
from pathlib import Path
import time

import cv2
import numpy as np
from ultralytics import YOLO

#PATHS
downloads   = Path.home() / "Downloads"
VIDEO_PATH  = downloads / "videos" / "car1.mp4"
OUTPUT_PATH = downloads / "car1_CSRT_final_output.mp4"

#HYPER‑PARAMETERS
CONF_THRESHOLD  = 0.05
IOU_THRESH      = 0.50     # skip duplicate tracks if IoU > this
HIST_BINS       = (16,16,16)
MAX_LOST_FRAMES = 60       # keep predicting this many missed frames

def extract_hsv_hist(patch, bins=HIST_BINS):
    hsv  = cv2.cvtColor(patch, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv],[0,1,2],None,bins,[0,180,0,256,0,256])
    cv2.normalize(hist, hist)
    return hist.flatten()

def iou(a, b):
    xa,ya = max(a[0],b[0]), max(a[1],b[1])
    xb,yb = min(a[2],b[2]), min(a[3],b[3])
    inter = max(0, xb-xa) * max(0, yb-ya)
    if inter == 0: return 0.0
    area_a = (a[2]-a[0])*(a[3]-a[1])
    area_b = (b[2]-b[0])*(b[3]-b[1])
    return inter / (area_a + area_b - inter)
class Track:
    def __init__(self, bbox_xywh, label, frame):
        self.tracker = cv2.TrackerCSRT_create()
        self.tracker.init(frame, bbox_xywh)

        x,y,w,h = map(int, bbox_xywh)
        self.bbox  = [x,y,w,h]
        self.label = label
        self.lost  = 0

        self.feature = extract_hsv_hist(frame[y:y+h, x:x+w])

        # Kalman state (cx,cy,vx,vy)
        cx, cy = x + w/2, y + h/2
        self.kalman = cv2.KalmanFilter(4,2)
        self.kalman.transitionMatrix   = np.array(
            [[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]], np.float32)
        self.kalman.measurementMatrix  = np.eye(2,4, dtype=np.float32)
        self.kalman.processNoiseCov    = np.eye(4, dtype=np.float32)*1e-2
        self.kalman.measurementNoiseCov= np.eye(2, dtype=np.float32)*1e-1
        self.kalman.statePre  = np.array([[cx],[cy],[0],[0]], np.float32)
        self.kalman.statePost = self.kalman.statePre.copy()

    def predict_center(self):
        p = self.kalman.predict()
        return float(p[0,0]), float(p[1,0])

    def correct(self, cx, cy):
        self.kalman.correct(np.array([[cx],[cy]], np.float32))

    def box_xyxy(self):
        x,y,w,h = self.bbox
        return [x, y, x+w, y+h]

model = YOLO("yolov8n.pt")

cap = cv2.VideoCapture(str(VIDEO_PATH))
assert cap.isOpened(), f"Cannot open {VIDEO_PATH}"
W, H   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps_in = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter(str(OUTPUT_PATH),
                      cv2.VideoWriter_fourcc(*"mp4v"),
                      fps_in, (W,H))

tracks      = []
prev_gray   = None
prev_pts    = None
frame_idx   = 0
start_time  = time.perf_counter()

feature_params = dict(maxCorners=200, qualityLevel=0.3,
                      minDistance=7, blockSize=7)
lk_params = dict(winSize=(15,15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT,
                           10, 0.03))

#MAIN LOOP
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1
    t0 = time.perf_counter()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #First frame: detect & spawn CSRT trackers
    if frame_idx == 1:
        det = model.predict(frame, conf=CONF_THRESHOLD,
                            classes=[0,3], verbose=False)[0]
        for box, cls in zip(det.boxes.xyxy.cpu().numpy().astype(int),
                            det.boxes.cls.cpu().numpy().astype(int)):
            if any(tr.label == model.names[int(cls)] and
                   iou(box, tr.box_xyxy()) > IOU_THRESH for tr in tracks):
                continue
            x1,y1,x2,y2 = box
            tracks.append(Track((x1,y1,x2-x1,y2-y1),
                                model.names[int(cls)], frame))
        prev_gray = gray.copy()
        prev_pts  = cv2.goodFeaturesToTrack(prev_gray, mask=None,
                                            **feature_params)
    #Subsequent frames
    else:
        # 1) global motion compensation
        curr_pts, st, _ = cv2.calcOpticalFlowPyrLK(prev_gray, gray,
                                                  prev_pts, None, **lk_params)
        good_prev = prev_pts[st.flatten()==1]
        good_curr = curr_pts[st.flatten()==1]
        M = np.eye(2,3, dtype=np.float32)
        if len(good_prev) >= 6:
            M,_ = cv2.estimateAffinePartial2D(good_prev, good_curr,
                                              method=cv2.RANSAC)
        stab = cv2.warpAffine(frame, M, (W,H))

        # 2) update each track
        new_tracks = []
        for tr in tracks:
            pcx, pcy = tr.predict_center()
            ok, bbox = tr.tracker.update(stab)
            if ok:
                tr.lost = 0
                tr.bbox = bbox
                x,y,w,h = map(int,bbox)
                tr.correct(x + w/2, y + h/2)
            else:
                tr.lost += 1
                if tr.lost > MAX_LOST_FRAMES:
                    continue  # give up on this track
                # fabricate a bbox around predicted center
                w,h = tr.bbox[2], tr.bbox[3]
                x,y = int(pcx - w/2), int(pcy - h/2)
                tr.bbox = [x,y,w,h]

            # draw back onto original (unstabilised) frame
            x,y,w,h = tr.bbox
            pts  = np.array([[x,y], [x+w,y+h]], np.float32).reshape(-1,1,2)
            invM = cv2.invertAffineTransform(M)
            ox1,oy1, ox2,oy2 = cv2.transform(pts, invM).reshape(-1,2).flatten()
            cv2.rectangle(frame, (int(ox1),int(oy1)),
                          (int(ox2),int(oy2)), (0,255,0), 2)
            cv2.putText(frame, tr.label, (int(ox1),int(oy1)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
            cv2.circle(frame, (int(pcx),int(pcy)), 4, (0,0,255), -1)
            new_tracks.append(tr)
        tracks = new_tracks

    #FPS overlay
    fps = 1.0 / (time.perf_counter() - t0)
    cv2.putText(frame, f"FPS: {fps:.1f}",
                (10,30), cv2.FONT_HERSHEY_SIMPLEX,
                0.9, (0,255,255), 2)

    out.write(frame)
    prev_gray = gray.copy()
    prev_pts  = cv2.goodFeaturesToTrack(prev_gray, mask=None,
                                        **feature_params)

total = time.perf_counter() - start_time
print(f"Processed {frame_idx} frames in {total:.2f}s — "
      f"Avg FPS: {frame_idx/total:.2f}")
print(f"Output saved to: {OUTPUT_PATH}")

cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
#KCF_tracker
import cv2
import torch
import numpy as np
from torchvision import models, transforms
from pathlib import Path
import uuid
import time
#PATHS
downloads = Path.home() / "Downloads"
VIDEO_PATH = downloads / "videos" / "car1.mp4"
OUTPUT_PATH = downloads / "car1_kcf_final_output.mp4"

#CONSTANTS
CONF_TH = 0.6
MAX_LOST = 180
HIST_THRESHOLD = 0.5
HIST_BINS = (16, 16, 16)

#COCO LABELS
COCO = {i: name for i, name in enumerate([
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', '', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', '', 'backpack', 'umbrella', '', '',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
    'tennis racket', 'bottle', '', 'wine glass', 'cup', 'fork', 'knife',
    'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli',
    'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
    'potted plant', 'bed', '', 'dining table', '', '', 'toilet', '', 'tv',
    'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
    'oven', 'toaster', 'sink', 'refrigerator', '', 'book', 'clock', 'vase',
    'scissors', 'teddy bear', 'hair drier', 'toothbrush'])}

def get_histogram(patch, bins=HIST_BINS):
    hsv = cv2.cvtColor(patch, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

def compare_hist(hist1, hist2):
    return cv2.compareHist(hist1.astype(np.float32), hist2.astype(np.float32), cv2.HISTCMP_CORREL)

class StrongKalmanFilter:
    def __init__(self, x, y):
        self.kf = cv2.KalmanFilter(8, 4)
        self.kf.transitionMatrix = np.eye(8, dtype=np.float32)
        for i in range(4):
            self.kf.transitionMatrix[i, i+4] = 1.0
        self.kf.measurementMatrix = np.zeros((4, 8), np.float32)
        for i in range(4):
            self.kf.measurementMatrix[i, i] = 1.0
        self.kf.processNoiseCov = np.eye(8, dtype=np.float32) * 1e-2
        self.kf.measurementNoiseCov = np.eye(4, dtype=np.float32) * 1e-1
        self.kf.errorCovPost = np.eye(8, dtype=np.float32)
        self.kf.statePost = np.array([[x], [y], [0], [0], [0], [0], [0], [0]], dtype=np.float32)

    def predict(self):
        pred = self.kf.predict()
        return int(pred[0]), int(pred[1])

    def correct(self, x, y):
        self.kf.correct(np.array([[x], [y], [0], [0]], dtype=np.float32))

#MODEL
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
model.to(device).eval()
transform = transforms.Compose([transforms.ToTensor()])

#VIDEO
cap = cv2.VideoCapture(str(VIDEO_PATH))
assert cap.isOpened(), f"Could not open {VIDEO_PATH}"
W, H = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS) or 25
out = cv2.VideoWriter(str(OUTPUT_PATH), cv2.VideoWriter_fourcc(*'mp4v'), fps, (W, H))

tracks = []
frame_idx = 0
t0 = prev = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1

    if frame_idx == 1:
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_tensor = transform(rgb).to(device)
        with torch.no_grad():
            preds = model([img_tensor])[0]

        boxes = preds['boxes'].cpu().numpy()
        labels_raw = preds['labels'].cpu().numpy()
        scores = preds['scores'].cpu().numpy()
        keep = scores >= CONF_TH
        boxes, labels_raw = boxes[keep], labels_raw[keep]

        for box, cls_id in zip(boxes, labels_raw):
            x1, y1, x2, y2 = map(int, box)
            w, h = x2 - x1, y2 - y1
            patch = frame[y1:y2, x1:x2]
            if patch.size == 0:
                continue
            tracker = cv2.TrackerKCF_create()
            tracker.init(frame, (x1, y1, w, h))
            hist = get_histogram(patch)
            kalman = StrongKalmanFilter(x1 + w//2, y1 + h//2)
            tracks.append({
                "id": str(uuid.uuid4())[:8],
                "tracker": tracker,
                "label": COCO.get(int(cls_id), f"class{cls_id}"),
                "hist": hist,
                "lost": 0,
                "kalman": kalman
            })
    else:
        new_tracks = []
        for tr in tracks:
            ok, bbox = tr["tracker"].update(frame)
            x, y, w, h = map(int, bbox)
            x2, y2 = x + w, y + h

            if not ok or x < 0 or y < 0 or x2 > W or y2 > H:
                tr["lost"] += 1
                pred_x, pred_y = tr["kalman"].predict()
                if tr["lost"] <= MAX_LOST:
                    cv2.circle(frame, (pred_x, pred_y), 4, (0, 0, 255), -1)
                    new_tracks.append(tr)
                continue

            patch = frame[y:y+h, x:x+w]
            if patch.size > 0:
                hist_new = get_histogram(patch)
                similarity = compare_hist(tr["hist"], hist_new)
                if similarity < HIST_THRESHOLD:
                    tr["lost"] += 1
                    if tr["lost"] <= MAX_LOST:
                        new_tracks.append(tr)
                    continue
                else:
                    tr["lost"] = 0
                    tr["hist"] = 0.8 * tr["hist"] + 0.2 * hist_new
                    tr["kalman"].correct(x + w//2, y + h//2)

            cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, tr["label"], (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            new_tracks.append(tr)

        tracks = new_tracks

    #FPS Overlay
    now = time.time()
    fps_now = 1.0 / (now - prev + 1e-6)
    prev = now
    cv2.putText(frame, f"FPS: {fps_now:.1f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    out.write(frame)


cap.release()
out.release()
total_time = time.time() - t0
avg_fps = frame_idx / total_time
print(f"✅ Saved to: {OUTPUT_PATH}")
print(f"Average FPS: {avg_fps:.2f}")


In [ ]:
#KJF_Tracker
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from collections import deque
from collections import deque

# --- تنظیمات ---
draw_points = True
video_path = "person4.mp4"
yolo_model_path = "yolov8n.pt"
output_path = "tracked_output.mp4"

max_corners_list = [2000, 1500]
min_distance_list = [5, 7]

min_tracked_points_list = [60, 60]

max_history_list = [1, 1]  # برای هر جسم جدا
max_tracked_points_list = [1000, 1000]

manual_weights_list = [
    [0.2, 0.5],     # جسم اول (برای 3 فریم: 2 قدیمی + 1 جدید)
    [0.4, 0.4],     # جسم دوم
]

# --- پارامترهای فیلتر برای هر جسم (بر اساکئدئنئاعس ترتیب در ویدیو) ---
speed_k_list = [100, 100]
max_movement_list = [25, 35]
threshold_ratio_list = [2.08, 1.8]
# --- کلاس Kalman با شتاب متغیر ---


sift = cv2.SIFT_create()  # یا
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)  # برای ORB
# یا برای SIFT می تونی NORM_L2 استفاده کنی


def extract_and_match_features(img1_gray, img2_gray):
    # استخراج فیچر و دتکتور و دیسکریپتور
    kp1, des1 = orb.detectAndCompute(img1_gray, None)
    kp2, des2 = orb.detectAndCompute(img2_gray, None)
    if des1 is None or des2 is None:
        return [], [], []

    # مچینگ فیچرها
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)

    # استخراج نقاط مچ شده
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    return pts1, pts2, matches


class kalam:
    count = 0

    def __init__(self, box):
        self.id = kalam.count
        kalam.count += 1
        self.lost = 0
        self.fresh = True
        self.kf = cv2.KalmanFilter(10, 4)
        self.kf.measurementMatrix = np.zeros((4, 10), dtype=np.float32)
        self.kf.measurementMatrix[0, 0] = 1
        self.kf.measurementMatrix[1, 1] = 1
        self.kf.measurementMatrix[2, 6] = 1
        self.kf.measurementMatrix[3, 7] = 1

        dt = 1 / 30.0
        dt2 = 0.5 * dt * dt

        T = np.eye(10, dtype=np.float32)
        T[0, 2], T[0, 4] = dt, dt2
        T[1, 3], T[1, 5] = dt, dt2
        T[2, 4] = dt
        T[3, 5] = dt
        T[6, 8] = dt
        T[7, 9] = dt
        self.kf.transitionMatrix = T
        self.kf.processNoiseCov = np.diag([0.1]*10).astype(np.float32)
        self.kf.measurementNoiseCov = np.eye(4, dtype=np.float32) * 0.1

        cx = (box[0]+box[2])/2
        cy = (box[1]+box[3])/2
        w = box[2]-box[0]
        h = box[3]-box[1]
        state = np.array([cx, cy, 0, 0, 0, 0, w, h, 0, 0], dtype=np.float32)
        self.kf.statePost = state.reshape(-1, 1)
        self.last_bbox = self._state_to_bbox(state)

    def _state_to_bbox(self, state):
        cx, cy, _, _, _, _, w, h, _, _ = state
        x1 = int(cx - w/2)
        y1 = int(cy - h/2)
        x2 = int(cx + w/2)
        y2 = int(cy + h/2)
        return (x1, y1, x2, y2)

    def predict(self):
        pred = self.kf.predict().flatten()
        self.lost += 1
        self.fresh = False
        self.last_bbox = self._state_to_bbox(pred)
        return self.last_bbox


    def update(self, point):
        # اگر ورودی نقطه بود
        if len(point) == 2:
            cx, cy = point
            # عرض و ارتفاع از حالت قبلی
            state = self.kf.statePost.flatten()
            w = state[6]
            h = state[7]
            box = (cx - w/2, cy - h/2, cx + w/2, cy + h/2)
        elif len(point) == 4:
            box = point
        else:
            raise ValueError(
                "Input to update should be a point (x,y) or a box (x1,y1,x2,y2)")

        cx = (box[0] + box[2]) / 2
        cy = (box[1] + box[3]) / 2
        w = box[2] - box[0]
        h = box[3] - box[1]
        meas = np.array([cx, cy, w, h], dtype=np.float32).reshape(-1, 1)
        self.kf.correct(meas)
        self.lost = 0
        self.fresh = True
        state = self.kf.statePost.flatten()
        self.last_bbox = self._state_to_bbox(state)


# --- توابع کمکی ---


def filter_points_by_speed_distribution(new_points, old_points, k=1.0):

    if len(new_points) == 0 or len(old_points) == 0:
        return np.array([], dtype=bool)
    velocity = new_points - old_points

    speed = np.linalg.norm(velocity, axis=1)

    mean_speed = np.mean(speed)
    std_speed = np.std(speed)

    lower_bound = mean_speed - k * std_speed
    upper_bound = mean_speed + k * std_speed
    mask = (speed >= lower_bound) & (speed <= upper_bound)

    return mask


def filter_points_far_from_cluster(points, threshold_ratio):
    if len(points) <= 1:
        return np.ones(len(points), dtype=bool)
    mean_point = np.mean(points, axis=0)
    distances = np.linalg.norm(points - mean_point, axis=1)
    mask = np.ones(len(points), dtype=bool)
    for i, dist in enumerate(distances):
        others = np.delete(distances, i)
        if dist > threshold_ratio * np.mean(others):
            mask[i] = False
    return mask


def filter_outliers_by_movement(new_points, old_points, max_movement):
    if len(new_points) == 0 or len(old_points) == 0:
        return np.array([], dtype=bool)
    movement = np.linalg.norm(new_points - old_points, axis=1)
    return movement < max_movement


def get_distributed_keypoints(image_gray, grid_size=(4, 4), max_corners=2000, min_distance=5):
    h, w = image_gray.shape
    gh, gw = grid_size
    keypoints = []
    for i in range(gh):
        for j in range(gw):
            roi = image_gray[i*h//gh:(i+1)*h//gh, j*w//gw:(j+1)*w//gw]
            corners = cv2.goodFeaturesToTrack(
                roi,
                maxCorners=max_corners // (gh * gw),
                qualityLevel=0.05,
                minDistance=min_distance
            )
            if corners is not None:
                for c in corners:
                    cx, cy = c.ravel()
                    keypoints.append([cx + j*w//gw, cy + i*h//gh])
    if not keypoints:
        return None
    return np.array(keypoints, dtype=np.float32).reshape(-1, 1, 2)


def get_weighted_points(p_history, confidence, weights_manual=None):
    """
    ترکیب نقاط از تاریخچه با وزن‌های دستی + وزن فریم فعلی با confidence.
    - p_history: deque از آرایه‌های (N, 2)
    - confidence: وزن فریم فعلی (بین 0 و 1)
    - weights_manual: لیست وزن‌های اولیه برای فریم‌های گذشته (به جز آخرین فریم)
    """
    n = len(p_history)
    if n == 0:
        return np.empty((0, 2)), np.array([])

    if weights_manual is None or len(weights_manual) != n - 1:
        weights_manual = np.ones(n - 1)

    weights_manual = np.array(weights_manual, dtype=np.float32)
    weights_manual /= weights_manual.sum()  # مجموعش ۱

    points_all = []
    weights_all = []

    # فریم‌های گذشته با (1 - confidence)
    for i in range(n - 1):
        pts = p_history[i].reshape(-1, 2)
        if len(pts) == 0:
            continue
        w = (1 - confidence) * weights_manual[i]
        points_all.append(pts)
        weights_all.append(np.full((len(pts),), w))

    # فریم آخر با confidence
    pts = p_history[-1].reshape(-1, 2)
    if len(pts) > 0:
        points_all.append(pts)
        weights_all.append(np.full((len(pts),), confidence))

    if len(points_all) == 0:
        return np.empty((0, 2)), np.array([])

    all_pts = np.vstack(points_all)
    all_weights = np.hstack(weights_all)

    return all_pts, all_weights


# --- راه‌اندازی ---
cap = cv2.VideoCapture(video_path)
ret, first_frame = cap.read()
if not ret:
    raise ValueError("ویدیو باز نشد")

fps = cap.get(cv2.CAP_PROP_FPS)
h0, w0 = first_frame.shape[:2]
out = cv2.VideoWriter(
    output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w0, h0))
model = YOLO(yolo_model_path).to(
    'cuda' if torch.cuda.is_available() else 'cpu')
model.fuse()

results = model(first_frame, conf=0.1, classes=[0, 3], max_det=2)[0]
preds = results.boxes.xyxy.cpu().numpy().astype(int)


trackers = []
for idx, box in enumerate(preds):
    x1, y1, x2, y2 = box[:4]
    roi_gray = cv2.cvtColor(first_frame[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY)

    max_corners = max_corners_list[idx] if idx < len(
        max_corners_list) else 2000
    min_distance = min_distance_list[idx] if idx < len(
        min_distance_list) else 5

    keypoints = get_distributed_keypoints(
        roi_gray,
        max_corners=max_corners,
        min_distance=min_distance
    )

    min_tracked_points = min_tracked_points_list[idx] if idx < len(
        min_tracked_points_list) else 5
    if keypoints is None or len(keypoints) < min_tracked_points:
        print(f"No keypoints found for object {idx}.")
        continue

    keypoints[:, 0, 0] += x1
    keypoints[:, 0, 1] += y1

    speed_k = speed_k_list[idx] if idx < len(speed_k_list) else 8.0
    max_move = max_movement_list[idx] if idx < len(max_movement_list) else 30
    thresh_ratio = threshold_ratio_list[idx] if idx < len(
        threshold_ratio_list) else 2.12
    max_history = max_history_list[idx] if idx < len(max_history_list) else 1
    max_tracked_pts = max_tracked_points_list[idx] if idx < len(
        max_tracked_points_list) else 1000
    manual_weights = manual_weights_list[idx] if idx < len(
        manual_weights_list) else None

    trackers.append({
        'kf': kalam(box),
        'p0': keypoints,
        'active': True,
        'p_history': deque([keypoints.reshape(-1, 2)], maxlen=max_history),
        'filter_params': {
            'max_movement': max_move,
            'threshold_ratio': thresh_ratio,
            'speed_k': speed_k,
            'max_tracked_points': max_tracked_pts,
            'manual_weights': manual_weights
        },
        'idx': idx,
        'min_tracked_points': min_tracked_points_list[idx] if idx < len(min_tracked_points_list) else 5,
        'max_history': max_history_list[idx] if idx < len(max_history_list) else 1
    })


print(f"Initialized trackers: {len(trackers)}")
old_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)


# --- حلقه اصلی ---
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    for obj in trackers:
        if obj['active']:
            p1, st, err = cv2.calcOpticalFlowPyrLK(
                old_gray, frame_gray, obj['p0'], None,
                winSize=(15, 15), maxLevel=2,
                criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

            if p1 is None or st.sum() < min_tracked_points:
                obj['active'] = False
                continue

            good_new = p1[st == 1]
            good_old = obj['p0'][st == 1]

            params = obj['filter_params']
            mask_movement = filter_outliers_by_movement(
                good_new, good_old, params['max_movement'])
            mask_cluster = filter_points_far_from_cluster(
                good_new, params['threshold_ratio'])
            mask_speed = filter_points_by_speed_distribution(
                good_new, good_old, k=params['speed_k'])

            final_mask = mask_movement & mask_cluster & mask_speed

            good_new = good_new[final_mask]
            if len(good_new) < min_tracked_points:
                obj['active'] = False
                continue

            # باکس فعلی از نقاط باقی‌مانده
            obj['p_history'].append(good_new.reshape(-1, 2))
            # اعتماد به فریم فعلی
            confidence = min(1.0, len(good_new) /
                             obj['filter_params']['max_tracked_points'])

            # گرفتن نقاط وزن‌دار
            points_combined, weights = get_weighted_points(
                obj['p_history'],
                confidence,
                weights_manual=obj['filter_params']['manual_weights']
            )

            if len(points_combined) < min_tracked_points:
                obj['active'] = False
                continue

            # ساختن باکس از نقاط وزن‌دار
            x1, y1 = np.min(points_combined, axis=0).astype(int)
            x2, y2 = np.max(points_combined, axis=0).astype(int)
            obj['kf'].update((x1, y1, x2, y2))
            # فقط آخرین فریم برای Optical Flow
            obj['p0'] = good_new.reshape(-1, 1, 2)
        else:
            # پیش‌بینی موقعیت بعدی از کالمن (باکس)
            x1, y1, x2, y2 = obj['kf'].predict()

            # کراپ ROI فریم قبلی و فعلی طبق کالمن
            old_roi = old_gray[y1:y2, x1:x2]
            new_roi = frame_gray[y1:y2, x1:x2]

            # فیچر استخراج و مچینگ
            pts_old, pts_new, matches = extract_and_match_features(
                old_roi, new_roi)

            if len(pts_old) >= min_tracked_points:
                # آپدیت نقاط پیگیری با نقاط مچ شده + تصحیح مختصات ROI به تصویر اصلی
                good_new = pts_new + np.array([x1, y1], dtype=np.float32)
                obj['p0'] = good_new.reshape(-1, 1, 2)
                max_history = obj.get('max_history', 1)
                if 'p_history' in obj:
                    obj['p_history'].append(good_new.reshape(-1, 2))
                else:
                    obj['p_history'] = deque(
                        [good_new.reshape(-1, 2)], maxlen=max_history)

                obj['active'] = True

                # ساخت باکس جدید از نقاط
                x1_new, y1_new = np.min(good_new, axis=0).astype(int)
                x2_new, y2_new = np.max(good_new, axis=0).astype(int)

                # به‌روزرسانی کالمن با centroid یا باکس جدید
                centroid = np.mean(good_new, axis=0)
                obj['kf'].update(centroid)  # فرض بر وجود update کالمن

                # به‌روزرسانی مختصات باکس با نقاط جدید
                x1, y1, x2, y2 = x1_new, y1_new, x2_new, y2_new

            else:
                # اگر فیچر مچینگ نتونست تعداد کافی پیدا کنه، fallback به اپتیکال فلو
                p0_old = obj.get('p0', None)
                if p0_old is not None and len(p0_old) > 0:
                    p1, st, err = cv2.calcOpticalFlowPyrLK(
                        old_gray, frame_gray, p0_old, None,
                        winSize=(15, 15), maxLevel=2,
                        criteria=(cv2.TERM_CRITERIA_EPS |
                                  cv2.TERM_CRITERIA_COUNT, 10, 0.03)
                    )
                    if p1 is not None and st.sum() >= min_tracked_points:
                        good_new = p1[st == 1]
                        obj['p0'] = good_new.reshape(-1, 1, 2)
                        max_history = obj.get('max_history', 1)
                        if 'p_history' in obj:
                            obj['p_history'].append(good_new.reshape(-1, 2))
                        else:
                            obj['p_history'] = deque(
                                [good_new.reshape(-1, 2)], maxlen=max_history)
                        obj['active'] = True

                        x1_new, y1_new = np.min(good_new, axis=0).astype(int)
                        x2_new, y2_new = np.max(good_new, axis=0).astype(int)

                        centroid = np.mean(good_new, axis=0)
                        obj['kf'].update(centroid)

                        x1, y1, x2, y2 = x1_new, y1_new, x2_new, y2_new
                    else:
                        # اگر باز هم نشد، همون پیش‌بینی کالمن رو نگه دار
                        pass
                else:
                    # اگر نقطه قدیمی نیست، کالمن رو نگه دار
                    pass

        # --- رسم باکس و نقاط ---
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        if draw_points and obj.get('p0') is not None:
            for pt in obj['p0']:
                cv2.circle(frame, tuple(pt[0].astype(int)), 2, (0, 0, 255), -1)

    out.write(frame)
    old_gray = frame_gray.copy()


cap.release()
out.release()
cv2.destroyAllWindows()